In [ ]:
# Import dependencies
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp

%matplotlib inline

In [ ]:
# Initialize Google Earth Engine API
import ee

ee.Authenticate()
ee.Initialize()

In [ ]:
# Import the Folium library to display map overlays
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Load polygons from geojson 
import json
from pathlib import Path

# Load the polygons from a geojson file - this can be easily created from a .shp file with geopandas
geojson_file_path = Path('/path/to/geojson/file/with/polygons')

with geojson_file_path.open('r') as j:
    amz_2005_feature_collection_dict = json.load(j) 

In [ ]:
# Get the centroid of the first polygon
scale = 1000 # scale in meters
poly_geo_j = amz_2005_feature_collection_dict["features"][0]
poly_centroid = ee.Geometry.Polygon(poly_geo_j["geometry"]["coordinates"]).centroid()
# Area of interest is 1km sphere around the centroid of the polygon
aoi = poly_centroid.buffer(scale)

In [ ]:
# Get Sentinel-1 SAR aggregation in August 2020 for the selected aoi
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31')) 
                       .first() 
                       .clip(aoi))

In [ ]:
# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH')))

# Create the map object.
location = poly_centroid.coordinates().getInfo()[::-1] # Folium expects coordinate as lat long, but gee uses long lat (or the opposite?)
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# Add polygon to the map
poly_geo_j_folium = folium.GeoJson(data=poly_geo_j, style_function=lambda x: {'fillColor': 'blue'})
poly_geo_j_folium.add_to(m)

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)